In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
import network_design as nd
from scipy.stats import entropy
import copy
from itertools import combinations

In [30]:
def find_rows_with_positive_values(matrix, columns, degree):
    # Create a mask for positive values in each specified column
    mask = np.ones(matrix.shape[0], dtype=bool)
    for col,deg in zip(columns,degree):
        mask &= (matrix[:, col] >= deg)
    # Get indices where all conditions are satisfied
    return np.where(mask)[0]

def generate_smart_node_assignments(g,O):
    # initialize matrix
    X = np.zeros((g.number_of_nodes(),O.shape[0]))
    for i in range(g.number_of_nodes()):
        # Get neighbors
        neighbors = g.neighbors(i)
        labels = []
        for n in neighbors:
            try:
                labels.append(np.where(X[n] == 1)[0][0])
            except:
                continue
        labels = np.array(labels)
        uniq_labels = np.unique(labels)
        degrees = np.array([len(labels[labels == l]) for l in uniq_labels])
        viable_labels = find_rows_with_positive_values(O,uniq_labels,degrees)
        if len(viable_labels) == 0:
            l = np.random.choice(np.arange(O.shape[0]))
        else:
            l = np.random.choice(viable_labels)
        X[i,l] = 1
    return X

def mh(f,g,O,burn_in=100,init_T=1,max_iters=1000):
    """
    Perform metropolis-hastings
    
    Parameters:
        f: function to minimize
        A (ndarray) - adjacency matrix
        O (ndarray) - binding matrix
        burn_in (ndarray) - number of iterations for burn in period
        init_T (float) - initial temperature
        max_iters (int) - maximum number of iterations
        cooling_factor (float) - factor to change temperature
    
    Return:
        X (ndarray) - node labels
        energy (float) - energy of final labeling
        entropy (float) - entropy of labeling
    """
    energies = []
    A = nx.adjacency_matrix(g).toarray()
    # Create initial labeling
    X0 = nd.generate_node_assignments(A.shape[0],O.shape[0]).astype(int)
    E0 = -np.log(f(A,X0.sum(axis=0),O,X0))
    labels = np.arange(O.shape[0])
    for _ in range(burn_in):
        # Update node labeling
        # Choose node
        k = np.random.choice(list(g.nodes()))
        # Choose random labeling
        # l = np.random.choice(labels)
        # Check neighbors
        neighbors = g.neighbors(k)
        # Get labels of neighbors
        labels = np.array([np.where(X0[n] == 1)[0][0] for n in neighbors])

        uniq_labels = np.unique(labels)
        degrees = np.array([len(labels[labels == l]) for l in uniq_labels])
        # Get viable labels
        viable_labels = find_rows_with_positive_values(O,uniq_labels,degrees)
        if len(viable_labels) == 0:
            continue
        else:
            l = np.random.choice(viable_labels)
        
        X1 = copy.deepcopy(X0)
        X1[k] = np.zeros(X1.shape[1]).astype(int)
        X1[k,l] = 1

        # Calculate new energy
        E1 = -np.log(f(A,X1.sum(axis=0),O,X1))
        # if len(diff[diff>0]) > 0:
        #     E1 = 10e2
        diff_E = np.linalg.norm(E0 - E1)
        random_num = np.random.random()
        if random_num <= np.min([1,np.exp(diff_E / init_T)]):
            X0 = X1
            E0 = E1
        energies.append(E0)
    
    T = init_T
    for _ in range(max_iters):
        T *= 1/np.log(1+_)
        # Update node labeling
        # Choose node
         # Update node labeling
        # Choose node
        k = np.random.choice(list(g.nodes()))
        # Choose random labeling
        # l = np.random.choice(labels)
        # Check neighbors
        neighbors = g.neighbors(k)
        # Get labels of neighbors
        labels = np.array([np.where(X0[n] == 1)[0][0] for n in neighbors])

        uniq_labels = np.unique(labels)
        degrees = np.array([len(labels[labels == l]) for l in uniq_labels])
        # Get viable labels
        viable_labels = find_rows_with_positive_values(O,uniq_labels,degrees)
        if len(viable_labels) == 0:
            continue
        else:
            l = np.random.choice(viable_labels)
        
        X1 = copy.deepcopy(X0)
        X1[k] = np.zeros(X1.shape[1])
        X1[k,l] = 1

        # Calculate new energy
        E1 = -np.log(f(A,X1.sum(axis=0),O,X1))
        # if len(diff[diff>0]) > 0:
        #     E1 = 10e2
        diff_E = np.linalg.norm(E0 - E1)
        random_num = np.random.random()
        if random_num <= np.min([1,np.exp(diff_E / init_T)]):
            X0 = X1
            E0 = E1
        energies.append(E0)
        
    return X0, E0, energies

In [3]:
def canonical_energy(params):
    A, X, O = params
    _, P = nd.canonical_ensemble(X.sum(axis=0).astype(int),O,X,ret_P=True)
    probability = np.prod([P[i,j]**A[i,j]*(1-P[i,j])**(1-A[i,j]) for j in range(i,A.shape[0]) for i in range(A.shape[0])])
    return probability

In [6]:
true_X = nd.generate_node_assignments(10,3)
cap = nd.assign_particles(3,4)
O = nd.create_O(cap)
O

array([[3., 0., 1.],
       [0., 2., 2.],
       [1., 1., 2.]])

In [7]:
g = nd.new_soup_of_nodes(true_X,nd.create_capacity(true_X,cap))

In [8]:
A = nx.adjacency_matrix(g).toarray()

In [31]:
new_X, new_E, energies = mh(nd.probability,g,O,max_iters=10000)

/tmp/ipykernel_17507/1367158969.py:54: RuntimeWarning: divide by zero encountered in log
  E0 = -np.log(f(A,X0.sum(axis=0),O,X0))
/tmp/ipykernel_17507/1367158969.py:81: RuntimeWarning: divide by zero encountered in log
  E1 = -np.log(f(A,X1.sum(axis=0),O,X1))
/tmp/ipykernel_17507/1367158969.py:84: RuntimeWarning: invalid value encountered in scalar subtract
  diff_E = np.linalg.norm(E0 - E1)
/tmp/ipykernel_17507/1367158969.py:93: RuntimeWarning: divide by zero encountered in scalar divide
  T *= 1/np.log(1+_)
/tmp/ipykernel_17507/1367158969.py:120: RuntimeWarning: divide by zero encountered in log
  E1 = -np.log(f(A,X1.sum(axis=0),O,X1))
/tmp/ipykernel_17507/1367158969.py:123: RuntimeWarning: invalid value encountered in scalar subtract
  diff_E = np.linalg.norm(E0 - E1)


In [43]:
nd.probability(A,test_X.sum(axis=0).astype(int),O,test_X)

np.float64(0.0)

In [41]:
test_X = generate_smart_node_assignments(g,O)

In [42]:
test_X

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])